# Merging multiple bounding box files into one 

## Part 1: Introduction

This Jupyter Notebook is intended to process **regional data portal**. It will merge all boudning box files within the reigon into one file for both city and county level. 

## Part 2: Preparation

We will be using **Jupyter Notebook(anaconda 3)** to edit and run the script. Information on Anaconda installation can be found <a href='https://docs.anaconda.com/anaconda/install/'>here</a>. Please note that this script is running on Python 3.

To run this script you need:
- all related city and county bounding box files stored in **portalCode** folder
- directory path (**geojsons** folder > **portalCode** folder)

The script currently prints two GeoJSON files:
- **portalCode_City_bbox**.json
- **portalCode_County_bbox**.json

>Original created on Jan 31 2021<br>
@author: Yijing Zhou @YijingZhou33

## Part 3: Get Started

###  Step 1: Import modules

In [ ]:
import pandas as pd
import os
import geopandas as gpd
import json
from itertools import chain
from functools import reduce
import folium

### Step 2: Manual items to change

In [ ]:
###### Portal Code ######
code = '04f-01'

## all states within the region
mergedstates = ['Maryland', 'New Jersey', 'Pennsylvania', 'Delaware']

## Part 4: Build up merged GeoJSON schema

### Step 3: Merge multiple GeoJSON files into one

In [ ]:
def merge_JsonFiles(mergedstates, level):
    dflist = []
    rootpath = os.path.dirname(os.getcwd())
    for state in mergedstates:
        filepath = os.path.join(rootpath, 'geojsons', code, f'{state}_{level}_bbox.json')
        geojson = gpd.read_file(filepath)
        ## delete source files
        os.remove(filepath)
        dflist.append(geojson)
    
    ## merge dataframes created by GeoJSON files
    gdf_merged = reduce(lambda left, right: left.append(right), dflist)
    gdf_merged.to_file(os.path.join(rootpath, 'geojsons', code, f'{code}_{level}_bbox.json'), driver='GeoJSON')
    
    return gdf_merged

county_geojson = merge_JsonFiles(mergedstates, 'County')
city_geojson = merge_JsonFiles(mergedstates, 'City')

## Part 5: Inspect merged bounding box map

In [ ]:
print('> Making map...')
## change the location here to zoom to the center
m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 5)

## check if the county geojson file can be rendered properly
folium.GeoJson(county_geojson,
               tooltip = folium.GeoJsonTooltip(fields=('County', 'State'),
               aliases=('County', 'State')),
               show = True).add_to(m)

## check if the city geojson file can be rendered properly
folium.GeoJson(city_geojson,
               tooltip = folium.GeoJsonTooltip(fields=('City', 'County', 'State'),
               aliases=('City', 'County', 'State')),
               show = True).add_to(m)

m